As students are preparing for midterm exams and the virtual exam days are gone and forgotten, each department is struggling to select at least one hall out of $N$ halls in the university to hold their exams. But here's the thing:
  - Students from each department aren't really fond of sitting next to students from other departments and must be separated (for example CE students and CS students must not share the same hall for exams).
  - Each department likes certain halls and won't hold their exams in other halls.
  - Sadly, the exit doors in some of the halls are stuck and the hard working staff have provided some routes so that the students from specific halls need to walk through the other hall and exit after the examination. And because they would like to prevent cheating as much as possible, students in such halls cannot be from the same department.
Check out the example below to fully understand the situation.

## Example
Look at this picture carefully.<br>
<img src="Images/CSP_example.png" width="400"/>
  - There are $6$ halls and $3$ departments.
  - The halls which the deparments like are indicated (for example, CE exams can be held only at Hall 1, 4 and 6).
  - Students from Hall 1 should exit using the doors in Hall 2 (Note: they all exit there and won't continue to Hall 3). In this case, EE students cannot be in both Hall 1 and 2 (But of course, it's okay for them to be in Hall 1 and 3).

## Note
  - You _must_ implement and use AC-3 algorithm.
  - You will _probably_ need to utilize the heuristic algorithms you have learned (e.g. MRV and LCV) or consider nearly tree-structured graphs approach in order to pass all tests in an appropriate time.

## Input
  - The first line consists of $N$ (count of halls) and $M$ (count of departments).
  - In the following $M$ lines, the $i$ th line ($1 \le i \le M$) is a list of preferred halls for the $i$ th department (separated by space).
  - In the next line, $E$, the total number of exit constraints (edges in the previous graph) are given.
  - in each of the following $E$ lines, a pair of hall numbers (separated by space) is given (reminder: the direction is important).
$$1 \le N \le 50$$
$$1 \le M \le 50$$

### Sample Input
This sample describes the previous example (CE=1, EE=2, ME=3).
```
6 3
1 4 6
1 2 3 5 6
3 4 5
5
1 2
2 3
3 4
3 5
3 6
```

## Output
In the only line, print:
  - list of one appropriate assignment ordered by hall number (separated by space).
  - `NO` if there isn't.

### Sample Output
One possible solution to the example above is {Hall1=CE, Hall2=EE , Hall3=ME, Hall4=CE, Hall5=EE, Hall6=EE}. So:
```
1 2 3 1 2 2
```
In another example, If Hall3 was not an option for ME, then the answer would be:
```
NO
```
Because then hall 2 or 3 would be empty.

## Your code
**Note:** It's OK to change the signature for the given functions and the given structure is just a suggestion to help you with the implementation. (you can't remove or add any cells based on the **Rules** section)

In [ ]:
def MRV(unassigned,values):
    m = len(unassigned)
    min_val = len(values[unassigned[0]-1])
    best_var = unassigned[0]
    for i in range(1,m):
        val = len(values[unassigned[i]-1])
        if val < min_val:
            min_val = val
            best_var = unassigned[i]
    return best_var

def LCV(node,node_neighbours,node_values):
    m = len(node_values[node-1])
    k = len(node_neighbours)
    best_val = node_values[node-1][0]
    min_conflict = 0
    for i in range(k):
        if best_val in node_values[node_neighbours[i]-1]:
            min_conflict += 1 
    if min_conflict == 0 or m==1 :
        return best_val
    for j in range(1,m):
        val = node_values[node-1][j]
        conflict = 0
        for i in range(k):
            if val in node_values[node_neighbours[i]-1]:
                conflict += 1 
        if conflict < min_conflict:
            min_conflict = conflict
            best_val = val
    return best_val

In [ ]:
def ac_3(variable,values,cpt,neighbour,unassigned):
    queue = cpt.copy()
    while len(queue) != 0:
        x1,x2 = queue.pop(0)
        if len(values[x1-1]) == 0:
            return False,values
        if x1 in unassigned:  
            for v in values[x1-1]:
                if v in values[x2-1]:
                    if len(values[x2-1])==1:
                        values[x1-1].remove(v)
                        for k in range(len(neighbour[x1-1])):
                            queue.append([x1,neighbour[x1-1][k]])
        if len(values[x2-1]) == 0:
            return False,values                            
        if x2 in unassigned:  
            if len(values[x2-1]) == 0:
                return False,values
            for v in values[x2-1]:
                if v in values[x1-1]:
                    if len(values[x1-1])==1:
                        values[x2-1].remove(v) 
                        for k in range(len(neighbour[x2-1])):
                            queue.append([x2,neighbour[x2-1][k]])
    return True, values 

In [ ]:
def backtrack(variable,values,cpt,neighbour,unassigned,assigned):
    n = len(variable)
    m = len(assigned)
    if n == m:
        return assigned
    state, values = ac_3(variable,values,cpt,neighbour,unassigned)
    if state == False:
        return 'NO'
    var = MRV(unassigned,values)
    unassigned.remove(var)
    while len(values[var-1])>0:
        values_temp = values.copy()
        val = LCV(var,neighbour[var-1],values)
        assigned.append([var,val])
        values_temp[var-1] = [val]
        result = backtrack(variable,values_temp,cpt,neighbour,unassigned,assigned)
        if result != 'NO':
            return assigned
        values[var-1].remove(val)
        assigned.remove([var,val])
    return 'NO'

def backtracking_search(variable,values,cpt,neighbour):
    unassigned = variable.tolist()
    assigned = []
    result = backtrack(variable,values,cpt,neighbour,unassigned,assigned)
    if result == 'NO':
        return result
    Final_result = np.zeros(len(result))
    for i in range(len(result)):
        Final_result[result[i][0]-1] = result[i][1]
    Final_result = np.int32(Final_result)
    return Final_result 

In [ ]:
import Helper_codes.question2 as q2
import time
import numpy as np

TIME_LIMIT = 3

tests = q2.get_all_tests(prefix='q2_')
tests_passed = 0
for test in tests:
    n, m, m_next_lines, e, next_e_lines = q2.scan_test_input(test)
    variable = np.arange(1,n+1)
    values = []
    for i in range(1,n+1):
        my_list = []
        for j in range(m):
            if i in m_next_lines[j]:
                my_list.append(j+1)
        values.append(my_list) 
    neighbour = []
    for i in range(1,n+1):
        my_list = []
        for j in range(e):
            if i == next_e_lines[j][0]:
                my_list.append(next_e_lines[j][1]) 
            elif i == next_e_lines[j][1]:
                my_list.append(next_e_lines[j][0]) 
        neighbour.append(my_list) 
    start_time = time.time()
    result = backtracking_search(variable,values,next_e_lines,neighbour)
    total_time = time.time() - start_time
    if q2.is_result_valid(test, result) and total_time < TIME_LIMIT:
        tests_passed += 1
    else:
        print(f'test {test} failed. time elapsed= {total_time}')
print(f'Score = {tests_passed / len(tests) * 100}%')